# Actividad 7 (Regresión Logística)
## Luis Javier González Romero - A01702728

In [578]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [579]:
df = pd.read_csv('cuentas_credicel.csv',encoding='latin')
df

<ipython-input-579-b7f43e925989>:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cuentas_credicel.csv',encoding='latin')


,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,HDNAWQ3A,30,12/11/21 19:00,ZTE,Blade A3 2020 RO B,26S,1949.0,780.0,0.0,...,40.02,5000,26,NaN,23.0,0,0,Tantoyuca,Veracruz,AEVM980413MVZNDR02
1,4,M418YNR4,33,12/11/21 19:15,MOTOROLA,LTE XT2097-12 E7I POWER Azul,13S,2999.0,1050.0,0.0,...,35.01,6000,13,NaN,60.0,PUEBLA,PUE.,Heroica Puebla de Zaragoza,Puebla,ZALC610119MPLPRR07
2,5,SZ7V3NZT,37,13/11/21 14:13,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,0,74,NaN,58.0,TANTOYUCA,VER.,0,0,0
3,6,9I08UIQZ,38,13/11/21 15:45,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,25.01,7000,13,NaN,42.0,AMXTLAN PUE,NaN,Zacatlan,Puebla,SAMJ790115MPLNRS02
4,7,JMF4LKYU,44,13/11/21 16:10,MOTOROLA,G10 B,39S,4299.0,1075.0,0.0,...,25.01,7000,74,NaN,33.0,TANTOYUCA,VER.,Tantoyuca,Veracruz,TEHF880707HVZRRR07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,92KQVRIX,96502,18/04/23 18:45,Motorola,Moto E13,26S,3349.0,297.0,0.0,...,8.87,10000,0,97880,22.0,OXKUTZCAB,YUC,Oxkutzcab,Yucatan,PACT990804MYNCCH05
22731,22968,NWB2EA8J,96479,18/04/23 18:46,Motorola,Moto E22i,26S,4100.0,1209.0,0.0,...,29.49,5000,0,37295,27.0,LEON,GTO,Leon,Guanajuato,HEGB950620MGTRMN03
22732,22969,VDTRJ2W6,96442,18/04/23 18:47,Samsung,A13,26S,4849.0,1152.0,0.0,...,23.76,4750,0,20210,36.0,AGUASCALIENTES,AGS,Aguascalientes,Aguascalientes,GADF870205HASLZL07
22733,22970,PETLF56C,96536,18/04/23 18:49,Samsung,A53 5G,39S,9799.0,2799.0,0.0,...,28.56,7000,0,78434,53.0,SOLEDAD DE GRACIANO SANCHEZ,SLP,Soledad De Graciano Sanchez,San Luis Potosi,DELJ700417HSPLRS03


In [580]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   folio               22735 non-null  int64  
 1   tag                 22735 non-null  object 
 2   folio_solicitud     22735 non-null  int64  
 3   fecha               22735 non-null  object 
 4   marca               22735 non-null  object 
 5   modelo              22735 non-null  object 
 6   plazo               22735 non-null  object 
 7   precio              22735 non-null  float64
 8   enganche            22735 non-null  float64
 9   descuento           22735 non-null  float64
 10  semana              22735 non-null  int64  
 11  monto_financiado    22735 non-null  float64
 12  costo_total         22735 non-null  int64  
 13  monto_accesorios    22735 non-null  float64
 14  agente_venta        22735 non-null  object 
 15  dis_venta           22735 non-null  object 
 16  stat

In [581]:
df.isnull().sum()

folio                    0
tag                      0
folio_solicitud          0
fecha                    0
marca                    0
modelo                   0
plazo                    0
precio                   0
enganche                 0
descuento                0
semana                   0
monto_financiado         0
costo_total              0
monto_accesorios         0
agente_venta             0
dis_venta                0
status                   0
fraude                   0
empresa                 13
inversion                0
pagos_realizados         0
reautorizacion           0
fecha_ultimo_pago     3356
fecha_pago_proximo    3356
status_cuenta         3314
puntos                   0
riesgo                   0
score_buro               0
razones_buro          7340
porc_eng                 0
limite_credito           0
semana_actual            0
cp_cliente            7243
edad_cliente             9
cd_cliente               0
edo_cliente              8
cd_venta                 0
e

In [582]:
df = df.fillna(df.mode().iloc[0])

In [583]:
df.isnull().sum()

folio                 0
tag                   0
folio_solicitud       0
fecha                 0
marca                 0
modelo                0
plazo                 0
precio                0
enganche              0
descuento             0
semana                0
monto_financiado      0
costo_total           0
monto_accesorios      0
agente_venta          0
dis_venta             0
status                0
fraude                0
empresa               0
inversion             0
pagos_realizados      0
reautorizacion        0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
puntos                0
riesgo                0
score_buro            0
razones_buro          0
porc_eng              0
limite_credito        0
semana_actual         0
cp_cliente            0
edad_cliente          0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
dtype: int64

In [584]:
y=df
percentile25=y.quantile(0.25) #Q1
percentile75=y.quantile(0.75) #Q3
iqr= percentile75 - percentile25

Limite_Superior= percentile75 + 1.5*iqr
Limite_Inferior= percentile25 - 1.5*iqr
print("Limite Superior: ", Limite_Superior)
print("Limite Inferior: ", Limite_Inferior)

Limite Superior:  folio                34333.500
folio_solicitud     139392.000
precio                7674.000
enganche              1841.750
descuento              500.000
semana                 457.500
monto_financiado      5703.000
costo_total          11108.500
monto_accesorios         0.000
status                   1.000
fraude                   0.000
inversion                0.000
pagos_realizados        34.500
reautorizacion           0.000
puntos                  20.000
riesgo                  60.300
score_buro            1370.000
porc_eng                36.825
semana_actual           48.500
edad_cliente            68.500
dtype: float64
Limite Inferior:  folio              -11290.500
folio_solicitud    -49692.000
precio                274.000
enganche             -104.250
descuento            -300.000
semana                 -2.500
monto_financiado      359.000
costo_total          -175.500
monto_accesorios        0.000
status                  1.000
fraude                  0.000

<ipython-input-584-76d2ed3a9723>:2: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  percentile25=y.quantile(0.25) #Q1
<ipython-input-584-76d2ed3a9723>:3: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  percentile75=y.quantile(0.75) #Q3


In [585]:
dfL= df[(y<=Limite_Superior)&(y>=Limite_Inferior)]
dfL

<ipython-input-585-967c8171ad5c>:1: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  dfL= df[(y<=Limite_Superior)&(y>=Limite_Inferior)]


,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,porc_eng,limite_credito,semana_actual,cp_cliente,edad_cliente,cd_cliente,edo_cliente,cd_venta,edo_venta,curp
0,3,NaN,30,NaN,NaN,NaN,NaN,1949.0,780.0,0.0,...,NaN,NaN,26.0,NaN,23.0,NaN,NaN,NaN,NaN,NaN
1,4,NaN,33,NaN,NaN,NaN,NaN,2999.0,1050.0,0.0,...,35.01,NaN,13.0,NaN,60.0,NaN,NaN,NaN,NaN,NaN
2,5,NaN,37,NaN,NaN,NaN,NaN,1959.0,490.0,0.0,...,25.01,NaN,NaN,NaN,58.0,NaN,NaN,NaN,NaN,NaN
3,6,NaN,38,NaN,NaN,NaN,NaN,1959.0,490.0,0.0,...,25.01,NaN,13.0,NaN,42.0,NaN,NaN,NaN,NaN,NaN
4,7,NaN,44,NaN,NaN,NaN,NaN,4299.0,1075.0,0.0,...,25.01,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,NaN,96502,NaN,NaN,NaN,NaN,3349.0,297.0,0.0,...,8.87,NaN,0.0,NaN,22.0,NaN,NaN,NaN,NaN,NaN
22731,22968,NaN,96479,NaN,NaN,NaN,NaN,4100.0,1209.0,0.0,...,29.49,NaN,0.0,NaN,27.0,NaN,NaN,NaN,NaN,NaN
22732,22969,NaN,96442,NaN,NaN,NaN,NaN,4849.0,1152.0,0.0,...,23.76,NaN,0.0,NaN,36.0,NaN,NaN,NaN,NaN,NaN
22733,22970,NaN,96536,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,28.56,NaN,0.0,NaN,53.0,NaN,NaN,NaN,NaN,NaN


In [586]:
dfL.isnull().sum()

folio                     0
tag                   22735
folio_solicitud           0
fecha                 22735
marca                 22735
modelo                22735
plazo                 22735
precio                  309
enganche                871
descuento                 0
semana                  946
monto_financiado        497
costo_total             787
monto_accesorios       1854
agente_venta          22735
dis_venta             22735
status                 5649
fraude                 1701
empresa               22735
inversion              5379
pagos_realizados        534
reautorizacion          774
fecha_ultimo_pago     22735
fecha_pago_proximo    22735
status_cuenta         22735
puntos                 7044
riesgo                   70
score_buro                0
razones_buro          22735
porc_eng                959
limite_credito        22735
semana_actual          1029
cp_cliente            22735
edad_cliente            283
cd_cliente            22735
edo_cliente         

In [587]:
dfL= df.drop(['tag', 'fecha', 'marca', 'modelo', 'plazo', 'agente_venta', 'dis_venta', 'empresa', 'fecha_ultimo_pago', 'fecha_pago_proximo', 'status_cuenta', 'razones_buro', 'limite_credito', 'cp_cliente', 'cd_cliente', 'edo_cliente', 'cd_venta', 'edo_venta', 'curp'], axis=1)
dfL

,folio,folio_solicitud,precio,enganche,descuento,semana,monto_financiado,costo_total,monto_accesorios,status,fraude,inversion,pagos_realizados,reautorizacion,puntos,riesgo,score_buro,porc_eng,semana_actual,edad_cliente
0,3,30,1949.0,780.0,0.0,81,1169.0,2106,0.0,2,0,0,0,0,0,0.00,0,40.02,26,23.0
1,4,33,2999.0,1050.0,0.0,225,1949.0,2925,0.0,2,0,0,0,0,0,0.00,0,35.01,13,60.0
2,5,37,1959.0,490.0,0.0,170,1469.0,2210,0.0,3,0,0,0,0,0,0.00,0,25.01,74,58.0
3,6,38,1959.0,490.0,0.0,170,1469.0,2210,0.0,2,0,0,0,0,0,0.00,0,25.01,13,42.0
4,7,44,4299.0,1075.0,0.0,178,3224.0,6942,0.0,1,0,0,0,0,0,0.00,0,25.01,74,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,96502,3349.0,297.0,0.0,211,3052.0,5486,0.0,1,0,0,0,0,10,27.40,469,8.87,0,22.0
22731,22968,96479,4100.0,1209.0,0.0,200,2891.0,5200,0.0,1,0,0,0,0,9,19.00,502,29.49,0,27.0
22732,22969,96442,4849.0,1152.0,0.0,256,3697.0,6656,0.0,1,0,0,0,0,11,26.65,459,23.76,0,36.0
22733,22970,96536,9799.0,2799.0,0.0,386,7000.0,15054,0.0,1,0,0,0,0,14,-17.35,699,28.56,0,53.0


In [588]:
dfL = dfL.fillna(df.mean())
dfL

<ipython-input-588-1d5d672020da>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  dfL = dfL.fillna(df.mean())


,folio,folio_solicitud,precio,enganche,descuento,semana,monto_financiado,costo_total,monto_accesorios,status,fraude,inversion,pagos_realizados,reautorizacion,puntos,riesgo,score_buro,porc_eng,semana_actual,edad_cliente
0,3,30,1949.0,780.0,0.0,81,1169.0,2106,0.0,2,0,0,0,0,0,0.00,0,40.02,26,23.0
1,4,33,2999.0,1050.0,0.0,225,1949.0,2925,0.0,2,0,0,0,0,0,0.00,0,35.01,13,60.0
2,5,37,1959.0,490.0,0.0,170,1469.0,2210,0.0,3,0,0,0,0,0,0.00,0,25.01,74,58.0
3,6,38,1959.0,490.0,0.0,170,1469.0,2210,0.0,2,0,0,0,0,0,0.00,0,25.01,13,42.0
4,7,44,4299.0,1075.0,0.0,178,3224.0,6942,0.0,1,0,0,0,0,0,0.00,0,25.01,74,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,96502,3349.0,297.0,0.0,211,3052.0,5486,0.0,1,0,0,0,0,10,27.40,469,8.87,0,22.0
22731,22968,96479,4100.0,1209.0,0.0,200,2891.0,5200,0.0,1,0,0,0,0,9,19.00,502,29.49,0,27.0
22732,22969,96442,4849.0,1152.0,0.0,256,3697.0,6656,0.0,1,0,0,0,0,11,26.65,459,23.76,0,36.0
22733,22970,96536,9799.0,2799.0,0.0,386,7000.0,15054,0.0,1,0,0,0,0,14,-17.35,699,28.56,0,53.0


In [589]:
dfL.isnull().sum()

folio               0
folio_solicitud     0
precio              0
enganche            0
descuento           0
semana              0
monto_financiado    0
costo_total         0
monto_accesorios    0
status              0
fraude              0
inversion           0
pagos_realizados    0
reautorizacion      0
puntos              0
riesgo              0
score_buro          0
porc_eng            0
semana_actual       0
edad_cliente        0
dtype: int64

In [590]:
n_col=df[['tag', 'fecha', 'marca', 'modelo', 'plazo', 'agente_venta', 'dis_venta', 'empresa', 'fecha_ultimo_pago', 'fecha_pago_proximo', 'status_cuenta', 'razones_buro', 'limite_credito', 'cp_cliente', 'cd_cliente', 'edo_cliente', 'cd_venta', 'edo_venta', 'curp']]
df = pd.concat([n_col, dfL], axis=1)
df

,tag,fecha,marca,modelo,plazo,agente_venta,dis_venta,empresa,fecha_ultimo_pago,fecha_pago_proximo,...,fraude,inversion,pagos_realizados,reautorizacion,puntos,riesgo,score_buro,porc_eng,semana_actual,edad_cliente
0,HDNAWQ3A,12/11/21 19:00,ZTE,Blade A3 2020 RO B,26S,Sintia Verenice Tenorio Perez E,mic0437,microtec,15/04/23 0:00,22/04/23 0:00,...,0,0,0,0,0,0.00,0,40.02,26,23.0
1,M418YNR4,12/11/21 19:15,MOTOROLA,LTE XT2097-12 E7I POWER Azul,13S,Alvaro Rafael Pascual,mic015,microtec,15/04/23 0:00,22/04/23 0:00,...,0,0,0,0,0,0.00,0,35.01,13,60.0
2,SZ7V3NZT,13/11/21 14:13,ZTE,BLADE A3 2020 Gris,13S,Sintia Verenice Tenorio Perez E,mic0437,microtec,15/04/23 0:00,22/04/23 0:00,...,0,0,0,0,0,0.00,0,25.01,74,58.0
3,9I08UIQZ,13/11/21 15:45,ZTE,BLADE A3 2020 Gris,13S,Luz Aylin Hernandez Lopez1,mic0748,microtec,15/04/23 0:00,22/04/23 0:00,...,0,0,0,0,0,0.00,0,25.01,13,42.0
4,JMF4LKYU,13/11/21 16:10,MOTOROLA,G10 B,39S,Sintia Verenice Tenorio Perez E,mic0437,microtec,05/03/22 0:00,12/03/22 0:00,...,0,0,0,0,0,0.00,0,25.01,74,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,92KQVRIX,18/04/23 18:45,Motorola,Moto E13,26S,Mildred Asuncion Arguello Caamal,svc044,servicel,18/04/23 18:45,25/04/23 0:00,...,0,0,0,0,10,27.40,469,8.87,0,22.0
22731,NWB2EA8J,18/04/23 18:46,Motorola,Moto E22i,26S,Maria Guadalupe Vargas Lopez,PXC004,pixelulares,18/04/23 18:46,25/04/23 0:00,...,0,0,0,0,9,19.00,502,29.49,0,27.0
22732,VDTRJ2W6,18/04/23 18:47,Samsung,A13,26S,Nicolás Jiménez Sánchez,gph008,microtec,18/04/23 18:47,25/04/23 0:00,...,0,0,0,0,11,26.65,459,23.76,0,36.0
22733,PETLF56C,18/04/23 18:49,Samsung,A53 5G,39S,Maribel Ulibarri Medina,gph014,grupoph,18/04/23 18:49,25/04/23 0:00,...,0,0,0,0,14,-17.35,699,28.56,0,53.0


In [591]:
df.isnull().sum()

tag                   0
fecha                 0
marca                 0
modelo                0
plazo                 0
agente_venta          0
dis_venta             0
empresa               0
fecha_ultimo_pago     0
fecha_pago_proximo    0
status_cuenta         0
razones_buro          0
limite_credito        0
cp_cliente            0
cd_cliente            0
edo_cliente           0
cd_venta              0
edo_venta             0
curp                  0
folio                 0
folio_solicitud       0
precio                0
enganche              0
descuento             0
semana                0
monto_financiado      0
costo_total           0
monto_accesorios      0
status                0
fraude                0
inversion             0
pagos_realizados      0
reautorizacion        0
puntos                0
riesgo                0
score_buro            0
porc_eng              0
semana_actual         0
edad_cliente          0
dtype: int64

In [592]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22735 entries, 0 to 22734
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tag                 22735 non-null  object 
 1   fecha               22735 non-null  object 
 2   marca               22735 non-null  object 
 3   modelo              22735 non-null  object 
 4   plazo               22735 non-null  object 
 5   agente_venta        22735 non-null  object 
 6   dis_venta           22735 non-null  object 
 7   empresa             22735 non-null  object 
 8   fecha_ultimo_pago   22735 non-null  object 
 9   fecha_pago_proximo  22735 non-null  object 
 10  status_cuenta       22735 non-null  object 
 11  razones_buro        22735 non-null  object 
 12  limite_credito      22735 non-null  object 
 13  cp_cliente          22735 non-null  object 
 14  cd_cliente          22735 non-null  object 
 15  edo_cliente         22735 non-null  object 
 16  cd_v

In [593]:
matriz = df.corr()
matriz

<ipython-input-593-7668110952fa>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  matriz = df.corr()


,folio,folio_solicitud,precio,enganche,descuento,semana,monto_financiado,costo_total,monto_accesorios,status,fraude,inversion,pagos_realizados,reautorizacion,puntos,riesgo,score_buro,porc_eng,semana_actual,edad_cliente
folio,1.000000,0.971332,0.098971,0.124622,0.130377,0.094659,0.075446,0.027103,0.081637,-0.381641,-0.051590,-0.139744,-0.477863,-0.267134,-0.120736,0.352654,0.548206,0.069327,-0.819061,0.047835
folio_solicitud,0.971332,1.000000,0.099934,0.121863,0.127074,0.097633,0.077342,0.029438,0.071465,-0.378045,-0.051017,-0.130940,-0.467299,-0.250896,-0.136859,0.340098,0.527624,0.063868,-0.796687,0.050773
precio,0.098971,0.099934,1.000000,0.678519,0.009883,0.672673,0.947080,0.872403,0.019097,-0.096183,0.060723,0.004445,0.017000,-0.057546,0.073077,-0.042888,0.080220,-0.011328,-0.030421,-0.017567
enganche,0.124622,0.121863,0.678519,1.000000,-0.025689,0.339912,0.408276,0.355694,0.031409,-0.032450,0.051645,-0.071798,-0.080730,-0.022209,-0.175815,0.272575,0.059764,0.676520,-0.094174,-0.013904
descuento,0.130377,0.127074,0.009883,-0.025689,1.000000,-0.026317,0.024136,0.036940,-0.006212,-0.076093,-0.035743,0.240813,0.040920,-0.016730,-0.087252,0.036555,0.103998,-0.050813,-0.112215,0.012407
semana,0.094659,0.097633,0.672673,0.339912,-0.026317,1.000000,0.688111,0.411911,0.028635,0.143145,0.028683,0.036724,-0.133589,0.006011,0.084821,-0.091175,0.072555,-0.158246,-0.189936,-0.018347
monto_financiado,0.075446,0.077342,0.947080,0.408276,0.024136,0.688111,1.000000,0.929859,0.030831,-0.108464,0.052949,0.036326,0.052593,-0.062734,0.165852,-0.169037,0.076027,-0.307681,-0.002362,-0.014597
costo_total,0.027103,0.029438,0.872403,0.355694,0.036940,0.411911,0.929859,1.000000,0.063101,-0.195627,0.060782,0.024614,0.132380,-0.086438,0.159682,-0.171733,0.060079,-0.305885,0.104572,-0.008468
monto_accesorios,0.081637,0.071465,0.019097,0.031409,-0.006212,0.028635,0.030831,0.063101,1.000000,-0.043031,0.029445,-0.050677,-0.062232,-0.026971,-0.040392,0.078120,0.039010,0.029359,-0.059894,0.034453
status,-0.381641,-0.378045,-0.096183,-0.032450,-0.076093,0.143145,-0.108464,-0.195627,-0.043031,1.000000,-0.155148,0.077710,0.251670,0.281953,0.062583,-0.143361,-0.240651,0.049292,-0.006862,-0.018282


In [594]:
matriz.n = np.abs(matriz)
matriz.n

<ipython-input-594-eedb82ff93a6>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  matriz.n = np.abs(matriz)


,folio,folio_solicitud,precio,enganche,descuento,semana,monto_financiado,costo_total,monto_accesorios,status,fraude,inversion,pagos_realizados,reautorizacion,puntos,riesgo,score_buro,porc_eng,semana_actual,edad_cliente
folio,1.000000,0.971332,0.098971,0.124622,0.130377,0.094659,0.075446,0.027103,0.081637,0.381641,0.051590,0.139744,0.477863,0.267134,0.120736,0.352654,0.548206,0.069327,0.819061,0.047835
folio_solicitud,0.971332,1.000000,0.099934,0.121863,0.127074,0.097633,0.077342,0.029438,0.071465,0.378045,0.051017,0.130940,0.467299,0.250896,0.136859,0.340098,0.527624,0.063868,0.796687,0.050773
precio,0.098971,0.099934,1.000000,0.678519,0.009883,0.672673,0.947080,0.872403,0.019097,0.096183,0.060723,0.004445,0.017000,0.057546,0.073077,0.042888,0.080220,0.011328,0.030421,0.017567
enganche,0.124622,0.121863,0.678519,1.000000,0.025689,0.339912,0.408276,0.355694,0.031409,0.032450,0.051645,0.071798,0.080730,0.022209,0.175815,0.272575,0.059764,0.676520,0.094174,0.013904
descuento,0.130377,0.127074,0.009883,0.025689,1.000000,0.026317,0.024136,0.036940,0.006212,0.076093,0.035743,0.240813,0.040920,0.016730,0.087252,0.036555,0.103998,0.050813,0.112215,0.012407
semana,0.094659,0.097633,0.672673,0.339912,0.026317,1.000000,0.688111,0.411911,0.028635,0.143145,0.028683,0.036724,0.133589,0.006011,0.084821,0.091175,0.072555,0.158246,0.189936,0.018347
monto_financiado,0.075446,0.077342,0.947080,0.408276,0.024136,0.688111,1.000000,0.929859,0.030831,0.108464,0.052949,0.036326,0.052593,0.062734,0.165852,0.169037,0.076027,0.307681,0.002362,0.014597
costo_total,0.027103,0.029438,0.872403,0.355694,0.036940,0.411911,0.929859,1.000000,0.063101,0.195627,0.060782,0.024614,0.132380,0.086438,0.159682,0.171733,0.060079,0.305885,0.104572,0.008468
monto_accesorios,0.081637,0.071465,0.019097,0.031409,0.006212,0.028635,0.030831,0.063101,1.000000,0.043031,0.029445,0.050677,0.062232,0.026971,0.040392,0.078120,0.039010,0.029359,0.059894,0.034453
status,0.381641,0.378045,0.096183,0.032450,0.076093,0.143145,0.108464,0.195627,0.043031,1.000000,0.155148,0.077710,0.251670,0.281953,0.062583,0.143361,0.240651,0.049292,0.006862,0.018282


In [595]:
df['status']= df['status'].replace([2,3,4], 0)
df['status'].value_counts()

1    17086
0     5649
Name: status, dtype: int64

In [596]:
df['marca'].value_counts()

Samsung         8991
SAMSUNG         3524
MOTOROLA        3078
ZTE             1948
Motorola        1658
NOKIA           1121
TECNO MOBILE     833
Hisense          371
HISENSE          270
Nokia            262
ALCATEL          215
TCL              113
Alcatel          101
Lanix             90
VIVO              60
Vivo              45
LG                20
WIKO               9
NYX                7
B2M                4
KODAK              3
RT                 3
LANIX              2
INCO               2
Huawei             1
SENWA              1
alcatel            1
Senwa              1
TECHPAD            1
Name: marca, dtype: int64

In [597]:
df['marca'] = df['marca'].replace(["SAMSUNG"], "Samsung")
df['marca'] = df['marca'].apply(lambda x: 1 if x == "Samsung" else (1 if x == "ZTE" else x))
df['marca'] = df['marca'].apply(lambda x: 0 if x != 1 else x)
df['marca'].value_counts()

1    14463
0     8272
Name: marca, dtype: int64

In [598]:
df['modelo'].value_counts()

A03s                             2231
A03                              1715
A13                              1089
A04                              1039
A04E                              817
                                 ... 
LTE 5002B 1B 32GB Verde             1
GALAXY A32 4+128 GB VIOLETA B       1
K62 B AZUL                          1
5007A 1V 2020 Verde                 1
4G 6002A 1V  Gris Iu                1
Name: modelo, Length: 366, dtype: int64

In [599]:
df['modelo'] = df['modelo'].apply(lambda x: 1 if x == "A03s" else (1 if x == "A03" else (1 if x == "A13" else (1 if x == "A04" else (1 if x == "A04E" else x)))))
df['modelo'] = df['modelo'].apply(lambda x: 0 if x != 1 else x)
df['modelo'].value_counts()

0    15844
1     6891
Name: modelo, dtype: int64

# Modelo 1

In [600]:
Vars_Indep= df[['pagos_realizados', 'precio', 'score_buro']]
Var_Dep= df['status']
X= Vars_Indep
y= Var_Dep

In [601]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [602]:
escalar = StandardScaler()

In [603]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [604]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [605]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [606]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 0])

In [607]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 362 1385]
 [ 270 4804]]


In [608]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.7762158668605591


In [609]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7573669549919366


In [610]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.946787544343713


In [611]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.8530586877386132


# Modelo 2


In [612]:
Vars_Indep= df[['pagos_realizados', 'precio', 'enganche']]
Var_Dep= df['status']
X= Vars_Indep
y= Var_Dep

In [613]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [614]:
escalar = StandardScaler()

In [615]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [616]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [617]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [618]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [619]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 221 1479]
 [ 190 4931]]


In [620]:
from sklearn.metrics import precision_score

precision2 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision2)

Precisión del modelo:
0.7692667706708268


In [621]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud2 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud2)

Exactitud del modelo:
0.7553144700190588


In [622]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad2 = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad2)

Sensibilidad del modelo:
0.9628978715094708


In [623]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.8552597346283931


# Modelo 3

In [624]:
Vars_Indep= df[['descuento', 'enganche', 'score_buro']]
Var_Dep= df['status']
X= Vars_Indep
y= Var_Dep

In [625]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [626]:
escalar = StandardScaler()

In [627]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [628]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [629]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [630]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [631]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0 1707]
 [   0 5114]]


In [632]:
from sklearn.metrics import precision_score

precision3 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision3)

Precisión del modelo:
0.7497434393783903


In [633]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud3 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud3)

Exactitud del modelo:
0.7497434393783903


In [634]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad3 = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad3)

Sensibilidad del modelo:
1.0


In [635]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.8569752827817344


# Modelo 4

In [636]:
Vars_Indep= df[['monto_financiado', 'precio', 'monto_accesorios']]
Var_Dep= df['status']
X= Vars_Indep
y= Var_Dep

In [637]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [638]:
escalar = StandardScaler()

In [639]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [640]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [641]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [642]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [643]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0 1693]
 [   0 5128]]


In [644]:
from sklearn.metrics import precision_score

precision4 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision4)

Precisión del modelo:
0.7517959243512682


In [645]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud4 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud4)

Exactitud del modelo:
0.7517959243512682


In [646]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad4 = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad4)

Sensibilidad del modelo:
1.0


In [647]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.858314503305716


# Modelo 5

In [648]:
Vars_Indep= df[['riesgo', 'porc_eng', 'score_buro']]
Var_Dep= df['status']
X= Vars_Indep
y= Var_Dep

In [649]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [650]:
escalar = StandardScaler()

In [651]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [652]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [653]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [654]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 0, 1, ..., 1, 1, 1])

In [655]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 281 1434]
 [ 299 4807]]


In [656]:
from sklearn.metrics import precision_score

precision5 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision5)

Precisión del modelo:
0.7702291299471239


In [657]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud5 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud5)

Exactitud del modelo:
0.745931681571617


In [658]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad5 = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad5)

Sensibilidad del modelo:
0.9414414414414415


In [659]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.8472724068035605


# Modelo 6

In [660]:
Vars_Indep= df[['pagos_realizados', 'precio', 'score_buro']]
Var_Dep= df['marca']
X= Vars_Indep
y= Var_Dep

In [661]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [662]:
escalar = StandardScaler()

In [663]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [664]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [665]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [666]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [667]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0 2454]
 [   0 4367]]


In [668]:
from sklearn.metrics import precision_score

precision6 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision6)

Precisión del modelo:
0.6402287054684064


In [669]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud6 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud6)

Exactitud del modelo:
0.6402287054684064


In [670]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad6 = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad6)

Sensibilidad del modelo:
1.0


In [671]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.7806578476939579


# Modelo 7

In [672]:
Vars_Indep= df[['monto_financiado', 'precio', 'monto_accesorios']]
Var_Dep= df['marca']
X= Vars_Indep
y= Var_Dep

In [673]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [674]:
escalar = StandardScaler()

In [675]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [676]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [677]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [678]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [679]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  28 2488]
 [  31 4274]]


In [680]:
from sklearn.metrics import precision_score

precision7 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision7)

Precisión del modelo:
0.6320615202602781


In [681]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud7 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud7)

Exactitud del modelo:
0.6306993109514734


In [682]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad7 = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad7)

Sensibilidad del modelo:
0.9927990708478514


In [683]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.7723863739044005


# Modelo 8

In [684]:
Vars_Indep= df[['riesgo', 'porc_eng', 'edad_cliente']]
Var_Dep= df['marca']
X= Vars_Indep
y= Var_Dep

In [685]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [686]:
escalar = StandardScaler()

In [687]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [688]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [689]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [690]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 0])

In [691]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  10 2508]
 [  20 4283]]


In [692]:
from sklearn.metrics import precision_score

precision8 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision8)

Precisión del modelo:
0.6306876748637903


In [693]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud8 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud8)

Exactitud del modelo:
0.6293798563260519


In [694]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad8 = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad8)

Sensibilidad del modelo:
0.9953520799442249


In [695]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.7721290787813233


# Modelo 9

In [696]:
Vars_Indep= df[['pagos_realizados', 'precio', 'reautorizacion']]
Var_Dep= df['modelo']
X= Vars_Indep
y= Var_Dep

In [697]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [698]:
escalar = StandardScaler()

In [699]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [700]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [701]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [702]:
y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 1, 0])

In [703]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4708   41]
 [2055   17]]


In [704]:
from sklearn.metrics import precision_score

precision9 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision9)

Precisión del modelo:
0.29310344827586204


In [705]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud9 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud9)

Exactitud del modelo:
0.6927136783462835


In [706]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad9 = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad9)

Sensibilidad del modelo:
0.008204633204633204


In [707]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.015962441314553988


# Modelo 10

In [708]:
Vars_Indep= df[['edad_cliente', 'precio', 'score_buro']]
Var_Dep= df['marca']
X= Vars_Indep
y= Var_Dep

In [709]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [710]:
escalar = StandardScaler()

In [711]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [712]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [713]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [714]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [715]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  10 2446]
 [  16 4349]]


In [716]:
from sklearn.metrics import precision_score

precision10 = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision10)

Precisión del modelo:
0.6400294334069169


In [717]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud10 = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud10)

Exactitud del modelo:
0.6390558569124761


In [718]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad10 = recall_score(y_test, y_pred, average="binary", pos_label=1)
print('Sensibilidad del modelo:')
print(sensibilidad10)

Sensibilidad del modelo:
0.9963344788087056


In [719]:
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score
puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label=1)
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.7793906810035843


# Tablas

In [720]:
coeficientes = pd.DataFrame({'Precision': [precision, precision2, precision3, precision4, precision5, precision6, precision7, precision8, precision9, precision10],
                             'Exactitud': [exactitud, exactitud2, exactitud3, exactitud4, exactitud5, exactitud6, exactitud7, exactitud8, exactitud9, exactitud10],
                             'Sensibilidad': [sensibilidad, sensibilidad2, sensibilidad3, sensibilidad4, sensibilidad5, sensibilidad6, sensibilidad7, sensibilidad8, sensibilidad9, sensibilidad10],})
indice = ['Modelo 1', 'Modelo 2', 'Modelo 3', 'Modelo 4', 'Modelo 5', 'Modelo 6', 'Modelo 7', 'Modelo 8', 'Modelo 9', 'Modelo 10']
coeficientes.index = indice
coeficientes

,Precision,Exactitud,Sensibilidad
Modelo 1,0.776216,0.757367,0.946788
Modelo 2,0.769267,0.755314,0.962898
Modelo 3,0.749743,0.749743,1.000000
Modelo 4,0.751796,0.751796,1.000000
Modelo 5,0.770229,0.745932,0.941441
Modelo 6,0.640229,0.640229,1.000000
Modelo 7,0.632062,0.630699,0.992799
Modelo 8,0.630688,0.629380,0.995352
Modelo 9,0.293103,0.692714,0.008205
Modelo 10,0.640029,0.639056,0.996334
